<a href="https://colab.research.google.com/github/clementsiegrist/segmentation_probabilisticDL/blob/main/bayesiannet_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget https://raw.githubusercontent.com/clementsiegrist/segmentation_probabilisticDL/main/requirements.txt 
!pip3 install -r requirements.txt
!pip3 install --upgrade --force-reinstall --no-deps kaggle

In [2]:
%%capture
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json


api_token = {"username":"bendidihab","key":"6b534264cbd90d4cc3f040b1894cb226"}
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)


!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c data-science-bowl-2018
!unzip /content/{/content}/competitions/data-science-bowl-2018/data-science-bowl-2018.zip
!mkdir /content/train
!mv -i /content/stage1_train.zip /content/train
!cd /content/train && unzip /content/train/stage1_train.zip
!mkdir /content/data && mkdir /content/data/train && mkdir /content/data/test
!rm -rf /content/stage1_sample_submission.csv.zip /content/stage1_solution.csv.zip /content/stage1_test.zip /content/stage1_train_labels.csv.zip /content/stage2_sample_submission_final.csv.zip /content/stage2_test_final.zip

In [3]:
import cv2 as cv
import os
import numpy as np
from google.colab.patches import cv2_imshow

In [4]:
i = 1
train_percent = 0.7
data_size = 670
dim = (640,480)
path = '/content/train'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
train_size = int(data_size*train_percent)
test_size = data_size - train_size
for sub in list_subfolders_with_paths :
  folder_types = [f.path for f in os.scandir(sub) if f.is_dir()]
  if i < train_size : 
    os.mkdir('data/train/OP'+str(i))
  else :
    os.mkdir('data/test/OP'+str(i))
  for f_type in folder_types :
    if f_type.split('/')[-1] == 'masks': 
      onlyfiles = [f for f in os.listdir(f_type) if os.path.isfile(os.path.join(f_type, f))]
      src = f_type + '/' + onlyfiles[0]
      src1 = cv.imread(src)
      for img in onlyfiles : 
        src2 = cv.imread(f_type + '/' + img)
        alpha = 1
        beta = 1
        src1 = cv.addWeighted(src1, alpha, src2, beta, 0.0)
      img = cv.resize(src1, dim, interpolation = cv.INTER_AREA)
      if i < train_size :
        cv.imwrite('data/train/OP'+str(i)+'/img_' + str(i) + '_class.png',img)
      else :
        cv.imwrite('data/test/OP'+str(i)+'/img_' + str(i) + '_class.png',img)
    if f_type.split('/')[-1] == 'images':
      onlyfiles = [f for f in os.listdir(f_type) if os.path.isfile(os.path.join(f_type, f))]
      img_raw = cv.imread(f_type + '/' + onlyfiles[0])
      img = cv.resize(img_raw, dim, interpolation = cv.INTER_AREA)
      if i < train_size :
        cv.imwrite('data/train/OP'+str(i)+'/img_' + str(i) + '_raw.png',img)
      else :
        cv.imwrite('data/test/OP'+str(i)+'/img_' + str(i) + '_raw.png',img)
  i += 1
!rm -rf /content/train /content/{

In [5]:
%%capture
!git clone https://github.com/yuta-hi/pytorch-trainer
!cd pytorch-trainer && python setup.py install
!git clone https://github.com/IhabBendidi/pytorch_bayesian_unet 
!cd pytorch_bayesian_unet && pip install -r requirements.txt && python setup.py install


!cp -r /content/data/train /content/pytorch_bayesian_unet/examples/cellular_segmentation/train
!cp -r /content/data/test /content/pytorch_bayesian_unet/examples/cellular_segmentation/test


In [6]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/preprocess.py

# train images: 467
100% 467/467 [00:00<00:00, 2668.57it/s]
# train labels: 467
100% 467/467 [00:02<00:00, 195.47it/s]
# test images: 203
100% 203/203 [00:00<00:00, 3048.09it/s]
# test labels: 203
100% 203/203 [00:01<00:00, 199.71it/s]


In [7]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py --iteration 3000

GPU: cuda:0
# Minibatch-size: 2

# classes: 2
# samples:
-- train: 421
-- valid: 46
iteration   main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
0                       1.85458e+08                          0.329972                  4.48171       
100         96528.3                           0.950257                                 44.7289       
     total [#.................................................]  3.33%
this epoch [#######################...........................] 47.51%
       100 iter, 0 epoch / 3000 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
200         56652.6                           0.942982                                 90.6839       
     total [###...............................................]  6.67%
this epoch [###############################################...] 95.01%
       200 iter, 0 epoch / 3000 iterations
     2.176 iters/sec. Estimated time to finish: 0:21:26.739137.
300         68264.6

In [8]:
!python /content/pytorch_bayesian_unet/examples/cellular_segmentation/train_and_test_epistemic.py --test_on_test

GPU: cuda:0
# Minibatch-size: 2

# samples:
-- test: 203
Loaded a snapshot: logs/predictor_iter_00003000.pth
